In [ ]:
%run params.ipynb
import rules.example

for name, rules in [('example', rules.example.build_rules())]:
    PARAMS['save_file'] = True
    PARAMS['name'] = name
    PARAMS['graph'] = rules['graph']          
    PARAMS['rules'] = rules['rules']        
    
    %run "01. structural analysis.ipynb"
    %run "02. build basins.ipynb"
    %run "03. analyse basins.ipynb"

In [37]:
import re
import rules.v01_oscillating
from rules.enzymes import *

all_nodes = {Actin_BR, Actin_ST, Arp2_3, CP, Cdc42, Cofilin, Coronin, Cortactin, 
             Dia1, Dia2, Ena_Vasp, IRSp53, LIMK, PAK, PIP2, PKD, Profilin, 
             ROCK, ROCK_, Rac1, RhoA, SSH, Thymosin, WASP, WAVE}

rules_map = {}
orig_rules = rules.v01_oscillating.build_rules() 
# rules_map["v01_oscillating"] = orig_rules

# knockdown nodes
for node in [Cdc42, PIP2]: #all_nodes - all_nodes:
    graph = orig_rules["graph"].copy()
    rules = orig_rules['rules']
    
    graph.remove_edges_from([(node, target) for target in list(graph[node])])      
    rules = re.sub('1:\s*'+node+'\*\s*=.+', '', rules)    
    rules = rules + '1: '+node+'* = False \n'
    
    rules_map["v01_oscillating_del_"+node] = {"graph": graph, "rules": rules}


# multi-knockdowns
for nodes in []: #[[Cdc42, Rac1], [Dia1, Dia2]]:
    graph = orig_rules["graph"].copy()    
    rules = orig_rules['rules']
    
    for node in nodes:
        graph.remove_edges_from([(node, target) for target in list(graph[node])])      
        rules = re.sub('1:\s*'+node+'\*\s*=.+', '', rules)    
        rules = rules + '1: '+node+'* = False \n'
              
    rules_map["v01_oscillating_del_"+"_".join(nodes)] = {"graph": graph, "rules": rules}
    
sorted(rules_map.keys())

# print(rules_map['v01_oscillating_del_Cdc42']['rules'])

['v01_oscillating_del_Cdc42', 'v01_oscillating_del_PIP2']

In [39]:
### run selected rules

%run params.ipynb

for name, rules in sorted(rules_map.items()):
    print("====== "+name)    
    PARAMS['name'] = name
    PARAMS['save_file'] = True
    PARAMS['graph'] = rules['graph']
    PARAMS['rules'] = rules['rules']
    PARAMS['chunk_size'] = 100
    PARAMS['chunks'] = 10
    PARAMS['report_basins'] = [[Actin_BR, Actin_ST]]
#     %run "01. structural analysis.ipynb"
#     %run "02. build basins.ipynb"
    %run "03. analyse basins.ipynb"

====== v01_oscillating_del_Cdc42
Normalizing cycles
Reporting simple basins
Reporting special basins
Reporting weighted activity
====== v01_oscillating_del_PIP2
Normalizing cycles
Reporting simple basins
Reporting special basins
Reporting weighted activity


In [ ]:
%run params.ipynb

arr = ['v01_oscillating', 'v01_oscillating_del_Arp2_3']

for name, rules in [(val, rules_map[val]) for val in arr]:
    print("====== "+name)    
    PARAMS['name'] = name
    PARAMS['save_file'] = True
    PARAMS['graph'] = rules['graph']
    PARAMS['rules'] = rules['rules']
    PARAMS['chunk_size'] = 2
    PARAMS['chunks'] = 5
    PARAMS['report_basins'] = [[Actin_BR, Actin_ST]]
    %run "01. structural analysis.ipynb"
    %run "02. build basins.ipynb"
    %run "03. analyse basins.ipynb"